In [1]:
import pandas as pd
import numpy as np

In [23]:
df=pd.read_csv('sampleAssigment.csv').drop('difficulty',axis=1)

In [47]:
df2=pd.read_csv('C:/research/adaptive learning/prereq_edges.csv')

In [51]:
def preprocess(students,prereq_edges):
    atomIndex={atom:i for i,atom in enumerate( students.atom_id.unique())}
    loIndex={lo:i for i,lo in enumerate( students.learning_objective_name.unique())}
    students['atomIndex']=students.atom_id.map(atomIndex)
    students['loIndex']=students.learning_objective_name.map(loIndex)
    
    #loMapping - learning objective mapping source to destination 
    loMapping=prereq_edges[(prereq_edges.source_lo_title.isin(loIndex.keys())) | (prereq_edges.dest_lo_title.isin(loIndex.keys()))]

    loMapping['source_index']=loMapping.source_lo_title.map(loIndex)
    loMapping['dest_index']=loMapping.dest_lo_title.map(loIndex)
        
    return students , loMapping


def train_test_split(df):
    df=preprocess(df)
    features=['user_id','atomIndex','correctBinary','difficulty','loIndex']
    users=df.user_id.unique()
    shuffledUsers=np.random.permutation(users)
    size=users.shape[0]
    train_users=shuffledUsers[:int(size*.8)]
    test_users=shuffledUsers[int(size*.8):]
    
    train_set=df[df.user_id.isin(train_users)]
    test_set=df[df.user_id.isin(test_users)]
    
    difficulty=train_set.groupby('atom_id').correctBinary.mean()
    
    train_set['difficulty']=train_set.atom_id.map(difficulty)
    test_set['difficulty']=test_set.atom_id.map(difficulty)
    
    train_set=train_set[features]
    test_set=test_set[features]
    
    return train_set,test_set


def create_train_graph():
    